In [2]:
!pip install praw


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [4]:
!pip install certifi


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import ssl
import certifi

# Force Python to use certifi's CA bundle
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())


In [2]:
import praw

# Create a Reddit instance (Replace with your credentials)
reddit = praw.Reddit(
    client_id="5XxI8AdauHxoDN8kgzck3w",
    client_secret="Y0PscnV3y40Kpz2kusRgnUPpB42Nww",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Function to fetch top posts and their top comments
def fetch_top_posts_with_comments(search_query, subreddit_name='all', post_limit=5, comment_limit=15):
    try:
        subreddit = reddit.subreddit(subreddit_name)
        top_posts = subreddit.search(search_query, sort='top', limit=post_limit)

        for post in top_posts:
            print(f"Post: {post.title}\n")
            
            # Fetch top comments
            post.comments.replace_more(limit=0)  # Load all comments
            top_comments = [comment.body for comment in post.comments.list()[:comment_limit]]
            
            for i, comment in enumerate(top_comments, 1):
                print(f"    Comment {i}: {comment}\n")
            print("\n------------------------\n")

    except Exception as e:
        print(f"Error fetching posts and comments: {e}")

# Main function
def main():
    search_query = input("Enter the topic you want to search for: ")
    subreddit_name = input("Enter the subreddit to search in (default is 'all'): ") or 'all'
    fetch_top_posts_with_comments(search_query, subreddit_name)

if __name__ == "__main__":
    main()


Post: It's a genuine question.

    Comment 1: At least for the PS5 Pro, WHY it exists is a relatively easy question to answer... Right?

WHY it exists specifically at THAT price though... Now that's the big question for Sony.

    Comment 2: >But why?

Because money.

    Comment 3: Why iphone 16 when you can wait another year and get iphone 17?

    Comment 4: It’s a filler episode year

    Comment 5: "We need your money, so I give you the same shit with slightly different but give me your money"

![gif](giphy|US86v9dnLweSaEQbq1|downsized)

    Comment 6: The iPhone 16 is just the yearly iPhone model, it exists as an incremental improvement over last year.

The PS5 Pro exists because money.

    Comment 7: ![gif](giphy|s239QJIh56sRW|downsized)

    Comment 8: Some diehard wanna-be rich technophobe kid :

![gif](giphy|am0dKpQO1vrOC3DmCb|downsized)

    Comment 9: Who’s the guy in the meme?

    Comment 10: People used to create great things and the selling point was that they'd last 

In [19]:
import praw
import json

# Create a Reddit instance (Replace with your credentials)
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Function to fetch top posts and their top comments
def fetch_top_posts_with_comments(search_query, subreddit_name='all', post_limit=5, comment_limit=15):
    data = []  # List to store post data
    try:
        subreddit = reddit.subreddit(subreddit_name)
        top_posts = subreddit.search(search_query, sort='top', limit=post_limit)

        for post in top_posts:
            post_data = {
                "title": post.title,
                "url": post.url,
                "comments": []
            }
            
            # Fetch top comments
            post.comments.replace_more(limit=0)  # Load all comments
            top_comments = [comment.body for comment in post.comments.list()[:comment_limit]]
            
            post_data["comments"].extend(top_comments)
            data.append(post_data)
        
        # Save data to a JSON file
        with open("reddit_posts.json", "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, indent=4, ensure_ascii=False)
        
        print("Data saved to reddit_posts.json")

    except Exception as e:
        print(f"Error fetching posts and comments: {e}")

# Main function
def main():
    search_query = input("Enter the topic you want to search for: ")
    subreddit_name = input("Enter the subreddit to search in (default is 'all'): ") or 'all'
    fetch_top_posts_with_comments(search_query, subreddit_name)

if __name__ == "__main__":
    main()

Data saved to reddit_posts.json


In [23]:
import praw
import requests
import spacy
import re
import ssl
from bs4 import BeautifulSoup

# Try to load Spacy model, use a fallback method if it fails
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic regex-based optimization.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)

        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts from the best subreddit
def fetch_reddit_posts(query):
    """Fetches top posts from the best subreddit for the query."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=5)

        results = []
        for post in top_posts:
            results.append({
                "title": post.title,
                "url": post.url,
                "comments": [comment.body for comment in post.comments[:5]]
            })
        
        return results
    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)

    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments']:
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


[WARNING] Spacy model not found. Using basic regex-based optimization.

🔍 Searching in subreddit: r/BudgetGamingLaptop for 'budget gaming laptop'


🔹 1. HOW IS THIS DEAL FOR A BUDGET GAMING LAPTOP?
🔗 URL: https://www.reddit.com/gallery/1ic181d
💬 Top Comments:
   - In Ca$ or Us$? In cad maybe but in usd NO!! I saw a RTX 4070m laptop for just 999 and with a far newer cpu
   - https://gaminglaptop.deals

🔹 2. Best Gaming Laptops in 2024/2025 List.
🔗 URL: https://www.reddit.com/r/BudgetGamingLaptop/comments/1emetuz/best_gaming_laptops_in_20242025_list/
💬 Top Comments:
   - [removed]
   - [removed]
   - Thanks for putting so much effort making this post. Definitely worth looking at it.

BTW, I use Predator and it's the best gaming laptop according to me, absolute beast.
   - I just picked up the Lenovo Legion 5 Pro with the AMD processor, and it’s a beast. It ticks all your boxes, but if you plan to game on it, you’ll likely need the power adapter. This is true for most gaming laptops.
   -

In [10]:
import praw

# Initialize Reddit API client
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Test if authentication works
print("Authentication successful:", reddit.read_only)  # Should print True


Authentication successful: True


In [24]:
import praw
import requests
import spacy
import re
import ssl
import json
from bs4 import BeautifulSoup

# Try to load Spacy model, use a fallback method if it fails
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic regex-based optimization.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)

        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts from the best subreddit
def fetch_reddit_posts(query):
    """Fetches top 10 posts from the best subreddit for the query and stores in a JSON file."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=10)

        results = []
        for post in top_posts:
            post.comments.replace_more(limit=0)  # Load all top-level comments
            comments = [comment.body for comment in post.comments[:20]]  # Get first 20 comments

            results.append({
                "title": post.title,
                "url": post.url,
                "comments": comments
            })
        
        # Save results to JSON file
        with open("reddit_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        print("✅ Data saved to reddit_results.json")

        return results

    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)

    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments'][:5]:  # Show only first 5 comments in the terminal
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


[WARNING] Spacy model not found. Using basic regex-based optimization.

🔍 Searching in subreddit: r/BudgetAudiophile for 'best gaming budget'

✅ Data saved to reddit_results.json

🔹 1. Best budget speakers against the wall
🔗 URL: https://www.reddit.com/r/BudgetAudiophile/comments/1ih26b3/best_budget_speakers_against_the_wall/
💬 Top Comments:
   - If you want them directly against the wall or maybe a few centimeters away, the only requirement is that they're not rear ported, as that would require merely a few inches, but not less.  Beyond that port spacing requirement, all speakers work best as as close to the wall as possible, but simply need EQ to tailor the bass response.  So buy whatever speakers you want based on the other factors you value or strengths they have, but you must implement a way of using parametric EQ, either from a PC, or miniDSP, or a source that has custom EQ ability or competent room correction.
   - You should look for front or downward ported speakers.   But eve

In [31]:
import praw
import requests
import spacy
import re
import ssl
import json
from bs4 import BeautifulSoup

# Try to load Spacy model, use a fallback method if it fails
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic regex-based optimization.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)

        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts and comments from the best subreddit
def fetch_reddit_posts(query):
    """Fetches top 10 posts and first 20 comments per post from the best subreddit."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=10)

        results = []
        for post in top_posts:
            post.comments.replace_more(limit=0)  # Load all top-level comments
            
            comments = []
            for comment in post.comments[:20]:  # Get first 20 comments
                comments.append(comment.body)

            results.append({
                "title": post.title,
                "url": post.url,
                "comments": comments
            })

        # Save results to JSON file
        with open("reddit_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        print("✅ Data saved to reddit_results.json")

        return results

    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)

    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments'][:5]:  # Show only first 5 comments in the terminal
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


[WARNING] Spacy model not found. Using basic regex-based optimization.

🔍 Searching in subreddit: r/RepTime for 'best gamint pc no budget'

✅ Data saved to reddit_results.json
⚠️ No relevant posts found.


In [30]:
import praw
import requests
import spacy
import json
import re
import ssl
from bs4 import BeautifulSoup

# Try to load Spacy model, use fallback if not available
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic keyword extraction.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue (If network blocks SSL requests)
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)

        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts and comments
def fetch_reddit_posts(query):
    """Fetches top 10 posts from the best subreddit and stores them with comments in a JSON file."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=10)

        results = []
        for post in top_posts:
            post.comments.replace_more(limit=0)  # Load all top-level comments
            comments = [comment.body for comment in post.comments.list()[:20]]  # Get first 20 comments

            results.append({
                "title": post.title,
                "url": post.url,
                "comments": comments
            })
        
        # Save results to JSON file
        with open("reddit_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        print("✅ Data saved to reddit_results.json")

        return results

    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)

    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments'][:5]:  # Show only first 5 comments in terminal
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


[WARNING] Spacy model not found. Using basic keyword extraction.

🔍 Searching in subreddit: r/gamingpc for 'best budget gaming pc'

✅ Data saved to reddit_results.json

🔹 1. Looking for the BEST speaker or soundbar for PC gaming purpose (movie and song sometime but not often) that FITs my limited space. No budget limit. I am considering Razer but heard it sucks. Is it real?
🔗 URL: https://www.reddit.com/gallery/1hpdibd
💬 Top Comments:
   - The simple answer  is to get your hands on some studio monitors ( speakers ). Run them through an external device  such as a focusrite solo amazing sound quality  for around 200
   - I've tried to like Razer, but everything I buy of theirs ends up failing or their software breaks things in my system. 2 mice, one keyboard and 2 sets of Leviathan v2, so I won't be doing that again. 

Feedback on the Leviathan v2 though: I ran the $200-300 model for about 6 months. When it worked, it sounded great. Not as good as a full 5 or 7.1 setup, but for its size 

In [32]:
!pip install praw requests spacy beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable


In [33]:
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/12.8 MB 1.3 MB/s eta 0:00:10
     --- ------------------------------------ 1.0/12.8 MB 1.4 MB/s eta 0:00:09
     ---- ----------------------------------- 1.6/12.8 MB 1.7 MB/s eta 0:00:07
     ------- -------------------------------- 2.4/12.8 MB 2.1 MB/s eta 0:00:06
     --------- ------------------------------ 2.9/12.8 MB 2.2 MB/s eta 0:00:05
     ------------ --------------------------- 3.9/12.8 MB 2.5 MB/s eta 0:00:04
     -------------- ------------------------- 4.7/12.8 MB 2.7 MB/s eta 0:00:03
     ------------------ --------------------- 5.8/12.8 MB 3.0 MB/s eta 0:00:03
     -------------------- ------------------- 6.6/12.8 MB 3.0 MB/s 

In [10]:
import praw
import requests
import spacy
import re
import ssl
import json
import torch
from bs4 import BeautifulSoup

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA-enabled GPU for processing.")
else:
    device = torch.device("cpu")
    print("CUDA not available, using CPU.")

# Try to load Spacy model, fallback to regex if unavailable
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("[WARNING] Spacy model not found. Using basic regex-based optimization.")
    nlp = None

# Reddit API Credentials
reddit = praw.Reddit(
    client_id="As44RYqjOK4m9FJ53EwC-g",
    client_secret="9jscdaZAODphthPzuVumkAgvH5EYJw",
    user_agent="Narative/0.1 by your_reddit_username"
)

# Fix SSL Issue
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# Step 1: Optimize the query
def optimize_query(query):
    """Optimizes user query using NLP or regex-based keyword extraction."""
    if nlp:
        doc = nlp(query)
        keywords = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
        return " ".join(keywords)
    else:
        # Fallback if Spacy is not installed
        query = query.lower()
        query = re.sub(r"[^a-zA-Z0-9 ]", "", query)  # Remove special characters
        keywords = query.split()  # Basic word split
        return " ".join(keywords)

# Step 2: Find the most relevant subreddit
def find_best_subreddit(query):
    """Finds the most relevant subreddit based on a given query."""
    search_url = f"https://www.reddit.com/search/?q={query}&type=sr"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        subreddit_links = soup.find_all('a', href=True)
        for link in subreddit_links:
            href = link['href']
            if '/r/' in href and not href.startswith("https://www.reddit.com/user/"):
                return href.split('/r/')[1].split('/')[0]  # Extract subreddit name
    return "all"  # Default to r/all if no specific subreddit is found

# Step 3: Fetch posts from the best subreddit
def fetch_reddit_posts(query):
    """Fetches top 10 posts from the best subreddit for the query and stores in a JSON file."""
    optimized_query = optimize_query(query)
    best_subreddit = find_best_subreddit(optimized_query)
    print(f"\n🔍 Searching in subreddit: r/{best_subreddit} for '{optimized_query}'\n")
    try:
        subreddit = reddit.subreddit(best_subreddit)
        top_posts = subreddit.search(optimized_query, limit=50)
        results = []
        for post in top_posts:
            post.comments.replace_more(limit=0)  # Load all top-level comments
            comments = [comment.body for comment in post.comments[:50]]  # Get first 50 comments
            results.append({
                "title": post.title,
                "url": post.url,
                "comments": comments
            })
        # Save results to JSON file
        with open("reddit_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        print("✅ Data saved to reddit_results.json")
        return results
    except Exception as e:
        print(f"⚠️ Error fetching posts: {e}")
        return []

# Step 4: Get user query and display results
if __name__ == "__main__":
    user_query = input("Enter your search query: ")
    posts = fetch_reddit_posts(user_query)
    if posts:
        for idx, post in enumerate(posts):
            print(f"\n🔹 {idx+1}. {post['title']}")
            print(f"🔗 URL: {post['url']}")
            print("💬 Top Comments:")
            for comment in post['comments'][:5]:  # Show only first 5 comments in the terminal
                print(f"   - {comment}")
    else:
        print("⚠️ No relevant posts found.")


Using CUDA-enabled GPU for processing.

🔍 Searching in subreddit: r/de for ''

✅ Data saved to reddit_results.json
⚠️ No relevant posts found.


In [1]:
import re
import json
import spacy
from collections import Counter
from transformers import pipeline

# Load NLP model for topic detection
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    nlp = None

# Define a helper function to clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s\.\,]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip()

# Extract key topics from text
def extract_topics(text, top_n=5):
    if nlp:
        doc = nlp(text)
        words = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
        return [word[0] for word in Counter(words).most_common(top_n)]
    return []  # Fallback if Spacy is unavailable

# Function to process and extract key content from JSON data
def preprocess_data(data):
    processed_text = []
    for entry in data:
        key_topics = extract_topics(entry["title"])  # Extract key topics from the title
        text_chunk = f"Title: {clean_text(entry['title'])}. "
        insightful_comments = []
        for comment in entry["comments"]:
            comment_lower = comment.lower().strip()
            if comment_lower in ["[deleted]", "[removed]", "thanks", "lol", "nice"]:
                continue  # Skip irrelevant comments
            insightful_comments.append(clean_text(comment))
        text_chunk += " ".join(insightful_comments[:5])  # Take top 5 meaningful comments
        processed_text.append((text_chunk, key_topics))
    return processed_text

# Splitting text into smaller chunks
def chunk_text(text, max_chunk_size=1024):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # +1 for space
        if current_length > max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word)
        else:
            current_chunk.append(word)
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

# Summarization function
def summarize_text(text_chunks, summarizer, max_length=200, min_length=50):
    summaries = []
    for chunk in text_chunks:
        summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)
        summaries.append(summary[0]["summary_text"])
    return summaries

# Load Reddit results
data_file = "reddit_results.json"
with open(data_file, "r", encoding="utf-8") as file:
    data = json.load(file)

# Pre-process text data
processed_data = preprocess_data(data)

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Generate summaries with key topics
final_summaries = []
for text, topics in processed_data:
    chunks = chunk_text(text)
    chunk_summaries = summarize_text(chunks, summarizer)
    full_summary = " ".join(chunk_summaries)
    if topics:
        full_summary = f"Key Topics: {', '.join(topics)}. {full_summary}"
    final_summaries.append(full_summary)

# Save final summaries
with open("reddit_summary.json", "w", encoding="utf-8") as f:
    json.dump(final_summaries, f, indent=4, ensure_ascii=False)

print("✅ Final summarized results saved to reddit_summary.json")


Device set to use cuda:0
Your max_length is set to 200, but your input_length is only 96. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)
Your max_length is set to 200, but your input_length is only 170. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)
Your max_length is set to 200, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 200, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer

✅ Final summarized results saved to reddit_summary.json


In [9]:
from transformers import pipeline

def summarize_key_points(filename):
    import json
    
    # Load the JSON file
    with open(filename, "r", encoding="utf-8") as file:
        data = json.load(file)

    combined_text = " ".join(data)

    # Limit the input text to 1024 tokens (approx. 4000 characters)
    combined_text = combined_text[:4000]  # Adjust if needed

    # Initialize summarization pipeline
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

    # Summarize in smaller chunks if text is too long
    summary = summarizer(combined_text, max_length=150, min_length=50, do_sample=False)

    return summary[0]["summary_text"]

# Example usage
summary_result = summarize_key_points("reddit_summary.json")
print("Final Summary:", summary_result)


Device set to use cpu


Final Summary: I feel Nirbhaya rapist were given capital punishment and matter closed,though it still took long,only because they were nobodies. Government controls the basic things like construction dust, seal the roads and regulate industries we can cut down pollution by 5070. Casteism and pollution, two of the biggest problems of India.
